In [26]:
# !pip install opencv-python-headless
# !pip install imageio
# !pip install OpenEXR
# !pip install Imath

import torchvision.models as models

import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"

import os
print(os.environ.get('OPENCV_IO_ENABLE_OPENEXR'))

import cv2
import imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
!pip install torchinfo
from torchinfo import summary

import numpy as np

import glob
from google.colab import drive
drive.mount('/content/drive/')

1
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [27]:
# !pip uninstall opencv-python-headless -y
# !pip uninstall opencv-python -y
# !apt update
# !apt install python3-opencv

In [28]:
import cv2
print(cv2.__version__)
print(cv2.getBuildInformation())

os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"

4.5.4

General configuration for OpenCV 4.5.4 =====================================
  Version control:               unknown

  Extra modules:
    Location (extra):            /build/opencv-AI3l5G/opencv-4.5.4+dfsg/contrib/modules
    Version control (extra):     unknown

  Platform:
    Timestamp:                   2022-12-05T08:00:00Z
    Host:                        Linux 5.4.0-135-generic x86_64
    CMake:                       3.22.1
    CMake generator:             Ninja
    CMake build tool:            /usr/bin/ninja
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2
      requested:                 SSE2
      disabled:                  SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2 AVX512_SKX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (14 files):         + SSE3 SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSE3 SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):      

In [29]:
#-------------------- LOAD DATASET HERE ----------------------------------
# !ls '/content/drive/MyDrive/hdr_dataset/hdr'
hdr_file_pattern = '/content/drive/MyDrive/hdr_dataset/hdr/*.exr'
# MyDrive/DIP Project/HDR Dataset 1'
# Loading one image:
hdr_images = []

for hdr_file_path in glob.glob(hdr_file_pattern):
    print(hdr_file_path)

# Now you have the image data in NumPy array format
# print("Image dimensions:", image_data.shape)

/content/drive/MyDrive/hdr_dataset/hdr/hdr_00000.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00001.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00038.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00026.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00030.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00019.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00008.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00020.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00035.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00025.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00024.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00011.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00006.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00015.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00022.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00032.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00002.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_00036.exr
/content/drive/MyDrive/hdr_dataset/hdr/hdr_000

In [30]:
# -------------------- IGNORE THIS BLOCK AT THE MOMENT --------------------

def discriminator_loss(D, real_images, fake_images):
  real_loss = F.mse_loss(D(real_images), torch.ones_like(D(real_images)))
  fake_loss = F.mse_loss(D(fake_images), torch.zeros_like(D(fake_images)))
  return real_loss + fake_loss

def generator_loss(G, fake_images):
  return F.mse_loss(G(fake_images), torch.ones_like(G(fake_images)))

# Optional: L1 Content Loss to enforce similarity between generated and real LDR images
def l1_content_loss(fake_images, real_images):
    return torch.mean(torch.abs(fake_images - real_images))


# def detector_loss():
#   # TBD
#   pass

In [31]:
# -------------------- BASIC GAN LOSSES -------------------------

def generator_adversarial_loss(D, fake_images):
    return torch.mean((D(fake_images) - 1) ** 2)

def discriminator_adverserial_loss(D, real_images, fake_images):
    real_loss = torch.mean((D(real_images) - 1) ** 2)
    fake_loss = torch.mean(D(fake_images) ** 2)
    return (real_loss + fake_loss) / 2

In [32]:
# ------------------------- GENERATOR ARCHITECTURE -------------------------

class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    super(ConvBlock, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
    self.ins = nn.InstanceNorm2d(out_channels)
    self.relu = nn.ReLU()

  def forward(self, x):
      x = self.conv(x)
      x = self.ins(x)
      x = self.relu(x)
      return x

class AttentionModule(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(AttentionModule, self).__init__()
    self.attention_score = nn.Conv2d(in_channels, in_channels, kernel_size=1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
      score = self.attention_score(x)
      score = self.sigmoid(score)
      attention_map = torch.mul(score, x)
      return attention_map

# Not using Attention Modules At the Moment

class Generator(nn.Module):
    def __init__(self, in_channels=3):
        super(Generator, self).__init__()
        self.conv1 = ConvBlock(in_channels, 32, kernel_size=1, stride=1, padding=0)
        self.conv2 = ConvBlock(32, 64, kernel_size=1, stride=1, padding=0)
        self.conv3 = ConvBlock(64, 128, kernel_size=1, stride=1, padding=0)
        self.conv4 = ConvBlock(512, 128, kernel_size=1, stride=1, padding=0)
        self.conv5 = ConvBlock(128, 64, kernel_size=1, stride=1, padding=0)
        self.conv6 = ConvBlock(64, 3, kernel_size=1, stride=1, padding=0)

        self.attention1 = AttentionModule(32, 32)
        self.attention2 = AttentionModule(64, 64)
        self.attention3 = AttentionModule(512, 512)

        self.k3 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.k5 = nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2)
        self.k7 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3)
        self.k9 = nn.Conv2d(128, 128, kernel_size=9, stride=1, padding=4)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest') # TESTING

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        # att_1 = self.attention1(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        # att_2 = self.attention2(x)
        x = self.conv3(x)
        x = self.maxpool(x)
        x_1 = self.k3(x)
        x_2 = self.k5(x)
        x_3 = self.k7(x)
        x_4 = self.k9(x)
        x = torch.cat((x_1, x_2, x_3, x_4), dim=1) # TESTING
        x = self.upsample(x) # CONTINUE CODING HERE
        x = self.conv4(x)
        x = self.upsample(x)
        x = self.conv5(x)
        x = self.upsample(x)
        x = self.conv6(x)
        return x


In [33]:
# --------------------- PATCH DISCRIMINATOR -------------------

class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(PatchDiscriminator, self).__init__()

        # Define layers
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1)
        self.conv5 = nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)

        # Instance normalization
        self.instance_norm = nn.InstanceNorm2d(512)

    def forward(self, x):
        # Forward pass through convolutional layers
        x = nn.LeakyReLU(0.2)(self.conv1(x))
        x = nn.LeakyReLU(0.2)(self.instance_norm(self.conv2(x)))
        x = nn.LeakyReLU(0.2)(self.instance_norm(self.conv3(x)))
        x = nn.LeakyReLU(0.2)(self.instance_norm(self.conv4(x)))
        x = self.conv5(x)

        return x

In [34]:
class Detector(nn.Module):
    def __init__(self):
        super(Detector, self).__init__()
        resnet = models.resnet50(pretrained=True)
        self.features = nn.Sequential(*list(resnet.children())[:-2])
        self.classifier = nn.Conv2d(2048, 20, kernel_size=1)  # Assuming 20 classes CHANGE ACCORDING TO CLASSES

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [35]:
# ---------------------------- TMO DET ----------------------

class TMODet(nn.Module):
    def __init__(self):
        super(TMODet, self).__init__()
        self.generator = Generator()
        self.discriminator = PatchDiscriminator()
        self.detector = Detector()

    def forward(self, x):
        tone_mapped = self.generator(x)
        detections = self.detector(tone_mapped)
        return tone_mapped, detections

In [36]:
H = 256 # CHANGE ACCORDING TO IMAGE HEIGHT
W = 256 # CHANGE ACCORDING TO IMAGE WIDTH
batch_size = 1
in_channels =3
# Testing

gen_model = Generator(in_channels=3)
print(summary(gen_model, input_size=(batch_size, in_channels, H, W)))

Layer (type:depth-idx)                   Output Shape              Param #
Generator                                [1, 3, 256, 256]          267,872
├─ConvBlock: 1-1                         [1, 32, 256, 256]         --
│    └─Conv2d: 2-1                       [1, 32, 256, 256]         128
│    └─InstanceNorm2d: 2-2               [1, 32, 256, 256]         --
│    └─ReLU: 2-3                         [1, 32, 256, 256]         --
├─MaxPool2d: 1-2                         [1, 32, 128, 128]         --
├─ConvBlock: 1-3                         [1, 64, 128, 128]         --
│    └─Conv2d: 2-4                       [1, 64, 128, 128]         2,112
│    └─InstanceNorm2d: 2-5               [1, 64, 128, 128]         --
│    └─ReLU: 2-6                         [1, 64, 128, 128]         --
├─MaxPool2d: 1-4                         [1, 64, 64, 64]           --
├─ConvBlock: 1-5                         [1, 128, 64, 64]          --
│    └─Conv2d: 2-7                       [1, 128, 64, 64]          8,320
│  

In [37]:
patch_disc_model = PatchDiscriminator(in_channels=3)
print(summary(patch_disc_model, input_size=(batch_size, in_channels, H, W)))

Layer (type:depth-idx)                   Output Shape              Param #
PatchDiscriminator                       [1, 1, 30, 30]            --
├─Conv2d: 1-1                            [1, 64, 128, 128]         3,136
├─Conv2d: 1-2                            [1, 128, 64, 64]          131,200
├─InstanceNorm2d: 1-3                    [1, 128, 64, 64]          --
├─Conv2d: 1-4                            [1, 256, 32, 32]          524,544
├─InstanceNorm2d: 1-5                    [1, 256, 32, 32]          --
├─Conv2d: 1-6                            [1, 512, 31, 31]          2,097,664
├─InstanceNorm2d: 1-7                    [1, 512, 31, 31]          --
├─Conv2d: 1-8                            [1, 1, 30, 30]            8,193
Total params: 2,764,737
Trainable params: 2,764,737
Non-trainable params: 0
Total mult-adds (G): 3.15
Input size (MB): 0.79
Forward/backward pass size (MB): 18.62
Params size (MB): 11.06
Estimated Total Size (MB): 30.47


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/instancenorm.py:80: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "


In [38]:
model = TMODet()
print(summary(model, input_size=(batch_size, in_channels, H, W)))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type:depth-idx)                             Output Shape              Param #
TMODet                                             [1, 3, 256, 256]          2,764,737
├─Generator: 1-1                                   [1, 3, 256, 256]          267,872
│    └─ConvBlock: 2-1                              [1, 32, 256, 256]         --
│    │    └─Conv2d: 3-1                            [1, 32, 256, 256]         128
│    │    └─InstanceNorm2d: 3-2                    [1, 32, 256, 256]         --
│    │    └─ReLU: 3-3                              [1, 32, 256, 256]         --
│    └─MaxPool2d: 2-2                              [1, 32, 128, 128]         --
│    └─ConvBlock: 2-3                              [1, 64, 128, 128]         --
│    │    └─Conv2d: 3-4                            [1, 64, 128, 128]         2,112
│    │    └─InstanceNorm2d: 3-5                    [1, 64, 128, 128]         --
│    │    └─ReLU: 3-6                              [1, 64, 128, 128]         --
│    └─MaxPool2d: 2

In [39]:
# stop here

SyntaxError: invalid syntax (<ipython-input-39-2f23345e57f1>, line 1)

In [40]:
def combined_loss(gen_loss, disc_loss, detection_loss, alpha=1.0, beta=1.0, gamma=1.0):
    """
    Compute the combined loss.

    Parameters:
    - gen_loss: Loss from the generator.
    - disc_loss: Loss from the discriminator.
    - detection_loss: Loss from the detector.
    - alpha, beta, gamma: Weighting factors for the generator loss, discriminator loss, and detection loss, respectively.

    Returns:
    - Total combined loss.
    """
    return alpha * gen_loss + beta * disc_loss + gamma * detection_loss

In [ ]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 100

discriminator_optimizer = torch.optim.Adam(model.discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
generator_optimizer = torch.optim.Adam(model.generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
detector_optimizer = torch.optim.Adam(model.detector.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Define loss functions
adversarial_loss = nn.BCEWithLogitsLoss()
classification_loss = nn.CrossEntropyLoss()
bbox_regression_loss = nn.MSELoss()

#--------------------------------------------------------------------------------
# dataloader is to be defined, we need to SETUP the dataset properly for our model
#--------------------------------------------------------------------------------

# Assuming you have a DataLoader named 'dataloader'
for epoch in range(num_epochs):
    for hdr_images, ldr_images, targets_boxes, targets_labels in dataloader:
        hdr_images = hdr_images.to(device)
        targets_boxes = targets_boxes.to(device)
        targets_labels = targets_labels.to(device)

        # Forward passes
        tone_mapped_images, detections = model(hdr_images)

        # Discriminator update
        discriminator_optimizer.zero_grad()
        real_loss = adversarial_loss(model.discriminator(ldr_images).view(-1), torch.ones_like(real))  #LDR IMAGES GO HERE
        fake_loss = adversarial_loss(model.discriminator(tone_mapped_images.detach()).view(-1), torch.zeros_like(fake))
        disc_loss = (real_loss + fake_loss) / 2

        # Generator update
        generator_optimizer.zero_grad()
        gen_loss = adversarial_loss(model.discriminator(tone_mapped_images).view(-1), torch.ones_like(fake))

        # Detector update - Placeholder for actual computation
        pred_boxes, pred_labels = detections  # Example decomposition
        detection_loss = classification_loss(pred_labels, targets_labels) + bbox_regression_loss(pred_boxes, targets_boxes)

        # Combined loss and backward
        total_loss = combined_loss(gen_loss, disc_loss, detection_loss, alpha=1.0, beta=1.0, gamma=1.0) # Can tune hyperparameters
        total_loss.backward()

        # Update steps
        discriminator_optimizer.step()
        generator_optimizer.step()
        detector_optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Total Loss: {total_loss.item()}')